In [1]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [29]:
import os
import sys; sys.path.insert(0, os.path.abspath("../"))
from sklearn.ensemble import GradientBoostingClassifier
import cv2
import xgboost as xgb
import numpy as np
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
import pandas as pd
from tqdm import tqdm
from dataset.dataset import SkinLesion_Dataset, SegExamples
from pipeline.preprocessing import SkinLesionPreprocessing
from pipeline.feature_extraction import FeaturesExtraction

import multiprocessing as mp
from tqdm import tqdm

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import pickle
from sklearn.svm import LinearSVC
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score, make_scorer, cohen_kappa_score, roc_auc_score, balanced_accuracy_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from boruta import BorutaPy

In [44]:
# Binary feather files
train_f = pd.read_feather("../data/binary/train_all_feat.f", columns=None, use_threads=True, storage_options=None)
val_f = pd.read_feather("../data/binary/val_all_features.f", columns=None, use_threads=True, storage_options=None)
train_shape = train_f.shape
val_shape = val_f.shape
print(f'train shape: {train_shape}, val shape: {val_shape}')
# Multi-class feather files.
train_f_mul = pd.read_feather("../data/three_class/train_all_feat.f", columns=None, use_threads=True, storage_options=None)
val_f_mul = pd.read_feather("../data/three_class/val_all_feat.f", columns=None, use_threads=True, storage_options=None)
train_shape_m = train_f_mul.shape
val_shape_m = val_f_mul.shape
print(f'train shape multi: {train_shape_m}, val shape multi: {val_shape_m}')

train shape: (15195, 144), val shape: (3796, 144)
train shape multi: (5082, 144), val shape multi: (1270, 144)


In [55]:
train_df_1 = pd.read_feather("../data/binary/train_all_feat.f")
train_df_2 = pd.read_feather("../data/binary/train_color_local_feat.f")
val_df_1 = pd.read_feather("../data/binary/val_all_features.f")
val_df_2 = pd.read_feather("../data/binary/val_color_local_feat.f")

train_df = pd.concat([train_df_2.iloc[:,:60], train_df_1], axis=1)
val_df = pd.concat([val_df_2.iloc[:, :60], val_df_1], axis=1)

train_shape = train_df.shape
val_shape = val_df.shape

train_df.replace({'nevus': 1, 'others': 0}, inplace=True)
val_df.replace({'nevus': 1, 'others': 0}, inplace=True)

df_train = train_df.iloc[:train_shape[0], :].sample(frac=1, random_state=42)
df_val = val_df.iloc[:val_shape[0], :].sample(frac=1, random_state=42)
# df_val

X_train, y_train = df_train.iloc[:, :(train_shape[1]-1)].to_numpy(dtype=np.float32), df_train.iloc[:, (train_shape[1]-1)].to_numpy()
X_test, y_test = df_val.iloc[:, :(val_shape[1]-1)].to_numpy(dtype= np.float32), df_val.iloc[:, (val_shape[1]-1)].to_numpy()

print(f'X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test: {y_test.shape}')
print(f'final class balance [TRAIN]: {np.unique(y_train, return_counts=True)}', f'[VAL]: {np.unique(y_test, return_counts=True)}')

X_train_f = pd.DataFrame(X_train)
X_test_f = pd.DataFrame(X_test)
y_train_f = pd.DataFrame(y_train)
y_test_f = pd.DataFrame(y_test)
frames = [X_train_f, X_test_f]
f = [y_train_f, y_test_f]
X = pd.concat(frames)
y = pd.concat(f)

# Feature Selection -- VARIANCE THRESHOLD
sel_var = VarianceThreshold(threshold=(.8 * (1 - .8)))
X_var= sel_var.fit_transform(X)
print(f'X_var: {X_var.shape}')

## Feature Selection -- SelectFromModel
sel_sfm = SelectFromModel(LinearSVC(C = 0.09, penalty="l1",  dual=False))
print("Performing Feature Selection ... ")
X_sfm = sel_sfm.fit_transform(X, y)
print(f'X_sfm: {X_sfm.shape}')

# split dataframe X again into train and test -- VARIANCE THRESHOLD
X_train_var = pd.DataFrame(X_var).iloc[:len(X_train_f[0])].to_numpy(dtype=np.float32)
X_test_var = pd.DataFrame(X_var).iloc[len(X_train_f[0]):].to_numpy(dtype=np.float32)
y_train_new = pd.DataFrame(y).iloc[:len(y_train_f[0])].to_numpy(dtype=np.float32)
y_test_new = pd.DataFrame(y).iloc[len(y_train_f[0]):].to_numpy(dtype=np.float32)
print(f'X_train_var: {X_train_var.shape}, y_train: {y_train.shape}, X_test_var: {X_test_var.shape}, y_test: {y_test.shape}')

# split dataframe X again into train and test SFM
X_train_new = pd.DataFrame(X_sfm).iloc[:len(X_train_f[0])].to_numpy(dtype=np.float32)
X_test_new = pd.DataFrame(X_sfm).iloc[len(X_train_f[0]):].to_numpy(dtype=np.float32)
y_train_new = pd.DataFrame(y).iloc[:len(y_train_f[0])].to_numpy(dtype=np.float32)
y_test_new = pd.DataFrame(y).iloc[len(y_train_f[0]):].to_numpy(dtype=np.float32)
print(f'X_train_new: {X_train_new.shape}, y_train: {y_train.shape}, X_test_new: {X_test_new.shape}, y_test: {y_test.shape}')

X_train: (15195, 203), y_train: (15195,), X_test: (3796, 203), y_test: (3796,)
final class balance [TRAIN]: (array([0, 1], dtype=int64), array([7470, 7725], dtype=int64)) [VAL]: (array([0, 1], dtype=int64), array([1865, 1931], dtype=int64))
X_var: (18991, 151)
Performing Feature Selection ... 


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


X_sfm: (18991, 133)
X_train_var: (15195, 151), y_train: (15195,), X_test_var: (3796, 151), y_test: (3796,)
X_train_new: (15195, 133), y_train: (15195,), X_test_new: (3796, 133), y_test: (3796,)


C:\Users\Lenovo\.virtualenvs\Skin_lesion_segmentation\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
# # Feature Selection -- Boruta Algorithm using XGboost
# model = RandomForestClassifier()
# feat_selector = BorutaPy(model, n_estimators ='auto', verbose=2, random_state=1, max_iter=5)
# feat_selector.fit(X_train, y_train)
# X_train_boruta = feat_selector.transform(X_train)
# X_test_boruta = feat_selector.transform(X_test)
# # print(f"best feature {feat_selector.ranking_}")
# print(f'X_train_boruta: {X_train_boruta.shape}, y_train: {y_train.shape}, X_test_boruta: {X_test_boruta.shape}, y_test: {y_test.shape}')

KeyboardInterrupt: 

In [8]:
# XGBoost -- without feature selection
param_grid = {'classifier__learning_rate': [0.01, 0.1],
              'classifier__n_estimators': [50, 100]}

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
classifier = xgb.XGBClassifier()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
grid_search = GridSearchCV(pipe, param_grid, cv=cv, refit = True, verbose = 2)
grid_search.fit(X_train, y_train)
print("TRAIN- The best parameters are %s with an accuracy of %0.4f"%(grid_search.best_params_, grid_search.best_score_))

Fitting 10 folds for each of 4 candidates, totalling 40 fits
[CV] END classifier__learning_rate=0.01, classifier__n_estimators=50; total time=  13.9s
[CV] END classifier__learning_rate=0.01, classifier__n_estimators=50; total time=  12.3s
[CV] END classifier__learning_rate=0.01, classifier__n_estimators=50; total time=  14.2s
[CV] END classifier__learning_rate=0.01, classifier__n_estimators=50; total time=  11.1s
[CV] END classifier__learning_rate=0.01, classifier__n_estimators=50; total time=  12.3s
[CV] END classifier__learning_rate=0.01, classifier__n_estimators=50; total time=  11.9s
[CV] END classifier__learning_rate=0.01, classifier__n_estimators=50; total time=  13.1s
[CV] END classifier__learning_rate=0.01, classifier__n_estimators=50; total time=  15.8s
[CV] END classifier__learning_rate=0.01, classifier__n_estimators=50; total time=  15.4s
[CV] END classifier__learning_rate=0.01, classifier__n_estimators=50; total time=  15.8s
[CV] END classifier__learning_rate=0.01, classifi

In [9]:
y_train_predicted = grid_search.predict(X_train)
y_test_predicted =  grid_search.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

F1 Score of train data: 0.9057 Acc: 0.9057584731819678
F1 Score of test data: 0.8099 Acc: 0.8100632244467861


In [53]:
# XGBOOST
param_grid = {'classifier__learning_rate': [0.01],
              'classifier__n_estimators': [300]
              }
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
classifier = xgb.XGBClassifier()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
grid_search = GridSearchCV(pipe, param_grid, cv=cv, refit = True, verbose = 3)
grid_search.fit(X_train, y_train)
print("TRAIN- The best parameters are %s with an accuracy of %0.4f"%(grid_search.best_params_, grid_search.best_score_))
y_train_predicted = grid_search.predict(X_train)
y_test_predicted =  grid_search.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV 1/2] END classifier__learning_rate=0.01, classifier__n_estimators=300;, score=0.813 total time=  56.9s
[CV 2/2] END classifier__learning_rate=0.01, classifier__n_estimators=300;, score=0.819 total time=  52.8s
TRAIN- The best parameters are {'classifier__learning_rate': 0.01, 'classifier__n_estimators': 300} with an accuracy of 0.8160
F1 Score of train data: 0.8675 Acc: 0.8675222112537019
F1 Score of test data: 0.8128 Acc: 0.8129610115911485


In [54]:
# Default xgboost without gridsearch
classifier = xgb.XGBClassifier()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train, y_train)
y_train_predicted = pipe.predict(X_train)
y_test_predicted =  pipe.predict(X_test)
print('F1 Score of train data: %0.4f' %f1_score(y_train,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test, y_test_predicted)}')

F1 Score of train data: 0.9923 Acc: 0.9923000987166831
F1 Score of test data: 0.8373 Acc: 0.8374604847207587


In [56]:
# REsults from Varianec threshold method
classifier = xgb.XGBClassifier()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train_var, y_train_new)
y_train_predicted = pipe.predict(X_train_var)
y_test_predicted =  pipe.predict(X_test_var)
print('F1 Score of train data: %0.4f' %f1_score(y_train_new,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train_new, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test_new,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test_new, y_test_predicted)}')

F1 Score of train data: 0.9895 Acc: 0.989470220467259
F1 Score of test data: 0.8239 Acc: 0.8240252897787145


In [57]:
# REsults from select from model method
classifier = xgb.XGBClassifier()
pipe = Pipeline([('scaler', StandardScaler()),('classifier', classifier)])
pipe.fit(X_train_new, y_train_new)
y_train_predicted = pipe.predict(X_train_new)
y_test_predicted =  pipe.predict(X_test_new)
print('F1 Score of train data: %0.4f' %f1_score(y_train_new,y_train_predicted,average='macro'), f'Acc: {accuracy_score(y_train_new, y_train_predicted)}')
print('F1 Score of test data: %0.4f' %f1_score(y_test_new,y_test_predicted,average='macro'), f'Acc: {accuracy_score(y_test_new, y_test_predicted)}')

F1 Score of train data: 0.9864 Acc: 0.9864429088515959
F1 Score of test data: 0.8263 Acc: 0.8263962065331928
